In [17]:
#Download Dataset

import downloader
downloader.get_drinks_dataset()
downloader.get_model()

Directory: data\ - exists
File: 		data\drinks.tar.gz
File Size:	146820533
Data Set Already Downloaded
Drinks Folder Already Exists


True

In [1]:
# import os
# #Get Model from Git
# pret_model_url = 'https://github.com/atsantiago2/dl_course_proj2.git'
# pret_model_dirname = 'git_pretrained_models'
# pret_model_filename = 'testfile.txt'
# pret_model_dirpath = os.path.join(pret_model_dirname, '')
# pret_model_path = os.path.join(pret_model_dirname, pret_model_filename)

# from git import Repo
# if os.path.isdir(pret_model_dirpath):
# 	print('Pretrained Model Already Downloaded Before')
# else:
# 	Repo.clone_from(pret_model_url, pret_model_dirpath)
# 	print('Pretrained Model Downloaded Already')

# #Copy Model to local
# import shutil
# if os.path.isfile(pret_model_filename):
# 	print('Pretrained Model Already in Place')
# else:
# 	shutil.copyfile(pret_model_filepath, pret_model_filename)
# 	print('Pretrained Model Copy Complete')


Pretrained Model Already Downloaded Before
Pretrained Model Already in Place


In [31]:

!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html --force-reinstall



Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp39-cp39-win_amd64.whl (3128.0 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
  Using cached numpy-1.22.3-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.0.1
    Uninstalling Pillow-9.0.1:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\duduy_work\\.conda\\envs\\dl_course\\lib\\site-packages\\pil\\_imaging.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:

print(torch.__version__, torch.cuda.is_available())

1.11.0 True


In [24]:

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model
# our dataset has two classes only - background and person
num_classes = 3


# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

model

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [25]:

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

[Parameter containing:
 tensor([[[[ 2.0535e-02]],
 
          [[ 1.2606e-02]],
 
          [[ 1.0740e-02]],
 
          ...,
 
          [[ 1.4291e-02]],
 
          [[-3.7358e-02]],
 
          [[ 7.9660e-03]]],
 
 
         [[[ 1.4908e-03]],
 
          [[-2.0951e-03]],
 
          [[-5.6496e-02]],
 
          ...,
 
          [[ 1.1562e-02]],
 
          [[ 9.4915e-03]],
 
          [[-4.6238e-02]]],
 
 
         [[[ 1.4271e-02]],
 
          [[ 4.6111e-05]],
 
          [[-2.2196e-02]],
 
          ...,
 
          [[ 1.3307e-02]],
 
          [[ 3.0431e-04]],
 
          [[-2.4062e-03]]],
 
 
         ...,
 
 
         [[[ 4.7004e-03]],
 
          [[ 9.1188e-03]],
 
          [[ 1.3270e-02]],
 
          ...,
 
          [[-5.3247e-03]],
 
          [[ 7.9189e-02]],
 
          [[ 8.2215e-03]]],
 
 
         [[[ 2.4644e-03]],
 
          [[ 2.4865e-03]],
 
          [[ 1.4478e-01]],
 
          ...,
 
          [[-5.8697e-03]],
 
          [[-2.6368e-02]],
 
          [[ 4.8512e-

In [15]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

'testfile.txt'